In [28]:
# Classification
import pandas as pd
import numpy as np
import FinanceDataReader as fdr
import yfinance as yf
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib
import pandas_ta
import talib
import datetime
import statsmodels.api as sm
from matplotlib import rc
%matplotlib inline

import warnings
warnings.filterwarnings('ignore')
from sklearn.model_selection import KFold, cross_val_score, train_test_split, StratifiedKFold
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier, VotingClassifier, StackingClassifier, AdaBoostClassifier, GradientBoostingClassifier
from sklearn.dummy import DummyClassifier
from sklearn.impute import SimpleImputer
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, confusion_matrix, roc_curve
from sklearn.pipeline import make_pipeline, Pipeline
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler, LabelEncoder, OrdinalEncoder, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.cluster import KMeans, AgglomerativeClustering
from sklearn.ensemble import RandomForestClassifier
# from lightgbm import LGBMClassifier
from xgboost import XGBClassifier
from catboost import CatBoostClassifier
import optuna
from sklearn.ensemble import HistGradientBoostingClassifier
from pykrx import stock
from scipy import stats

In [ ]:
# 한국 업종별 ETF 불러오기

kr_반도체 = fdr.DataReader('091160')
kr_은행 = fdr.DataReader('091170')
kr_철강 = fdr.DataReader('117680')
kr_건설 = fdr.DataReader('117700')
kr_증권 = fdr.DataReader('102970')
kr_에너지화학 = fdr.DataReader('117460')
kr_기계장비 = fdr.DataReader('102960')

kr_자동차 = fdr.DataReader('091180')
kr_IT = fdr.DataReader('266370')
kr_운송 = fdr.DataReader('140710')
kr_필수소비재 = fdr.DataReader('266410')
kr_경기소비재 = fdr.DataReader('266390')
kr_보험 = fdr.DataReader('140700')
kr_헬스케어 = fdr.DataReader('266420')

# 미국 업종별 ETF 불러오기
us_반도체 = fdr.DataReader('SOXX')
us_은행 = fdr.DataReader('XLF')
us_철강 = fdr.DataReader('XME')
us_건설 = fdr.DataReader('ITB')
us_증권 = fdr.DataReader('IYE')
us_에너지화학 = fdr.DataReader('XLE')
us_기계장비 = fdr.DataReader('XLI')

us_자동차 = fdr.DataReader('CARZ')
us_IT = fdr.DataReader('QQQ')
us_운송 = fdr.DataReader('XTN')
us_필수소비재 = fdr.DataReader('XLP')
us_경기소비재 = fdr.DataReader('XLY')
us_보험 = fdr.DataReader('KIE')
us_헬스케어 = fdr.DataReader('XLV')

In [19]:
ks = yf.download('^KS11', start='2000-01-01', end='2023-02-01')
ks200 = yf.download('^KS200', start='2000-01-01', end='2023-02-01')
kq = yf.download('^KQ11', start='2000-01-01', end='2023-02-01')
sp = yf.download('^GSPC', start='2000-01-01', end='2023-02-01')
nq = yf.download('^IXIC', start='2000-01-01', end='2023-02-01')
dow = yf.download('DOW', start='2000-01-01', end='2023-02-01')

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


### t-test 검증

In [2]:
# 각 연도별로 데이터 가져오기
def sep_date(data):
    santa = []
    for year in range(2000, 2023):
        try : 
            year_data = data[data.index.year == year]
            year_data_next = data[data.index.year == year + 1]

            # 연말 5일 전 계산
            end_of_year = year_data.tail(5).index[0]

            # 연초 2일 계산
            start_of_year = year_data_next.head(2).index[-1]

            # 연속된 데이터 추출
            continuous_data = data.loc[end_of_year:start_of_year]
#             print(continuous_data)
            rest_data = data[:end_of_year]
            # 결과 출력 또는 원하는 작업 수행
            santa.append(continuous_data)
        except:
            pass
    return santa

In [4]:
def make_1d_pct(df):
    df['1d_pct'] = ((df['Close'] - df['Open']) / df['Open']) * 100
    return df

In [5]:
def mannwhitneyu_test(df):
    santa_period = sep_date(df)
    santa_period = pd.concat(santa_period)
    rest_period = df[~df.index.isin(santa_period.index)]
    santa_period = make_1d_pct(santa_period)
    rest_period = make_1d_pct(rest_period)
    print(santa_period[['1d_pct', 'Volume']].describe())
    print(rest_period[['1d_pct', 'Volume']].describe())
    U_statistic, p_value = stats.mannwhitneyu(santa_period['1d_pct'], rest_period['1d_pct'])
    print(p_value)
    return p_value

In [82]:
# 각 연도별로 데이터 가져오기
def preemptive_reflection(data, start, end):
    santa = []
    rest = []
    for year in range(2000, 2023):
        try : 
            year_data = data[data.index.year == year]
            year_data_next = data[data.index.year == year + 1]
            
            # 선반영
            end_of_year = year_data.iloc[start:end,] # 1주일전
            rest_day = year_data.iloc[:start]
#             print(end_of_year)
            # 결과 출력 또는 원하는 작업 수행
            santa.append(end_of_year)
            rest.append(rest_day)
        except:
            pass
    return santa, rest

In [83]:
def mannwhitneyu_test_preemptive_reflection(df):
    reflection_range = [[-33, -26], [-26, -19], [-19, -12], [-12, -5]]
    p_value_list = []
    for i in reflection_range:
        santa_period, rest_period = preemptive_reflection(df, i[0], i[1])
        santa_period = pd.concat(santa_period)
        rest_period = pd.concat(rest_period)
        santa_period = make_1d_pct(santa_period)
        rest_period = make_1d_pct(rest_period)
        print(santa_period[['1d_pct', 'Volume']].describe())
        print(rest_period[['1d_pct', 'Volume']].describe())
        U_statistic, p_value = stats.mannwhitneyu(santa_period['1d_pct'], rest_period['1d_pct'])
        p_value_list.append(p_value)
        print(p_value)
    return p_value_list

In [8]:
kr_etf_list = ['091160', '091170', '117680', '117700', '102970', '117460', '102960', '091180', '266370', '140710', '266410', '266390', '140700', '266420']
us_etf_list = ['SOXX', 'XLF', 'XME', 'ITB', 'IYE', 'XLE', 'XLI', 'CARZ', 'QQQ', 'XTN', 'XLP', 'XLY', 'KIE', 'XLV']

In [84]:
kr_santa_list = []

for i in kr_etf_list:
    try : 
        print(f'---------------------------------------------------- {i} ----------------------------------------------------')
        df = fdr.DataReader(i, start='2000-01-01', end='2023-02-01')
        p_value1 = mannwhitneyu_test(df)
        p_value_list = mannwhitneyu_test_preemptive_reflection(df)
        if round(p_value1, 5) <= 0.05:
            kr_santa_list.append(i)
        else:
            for item in p_value_list:
                if round(item, 5) <= 0.05:
                    kr_santa_list.append(i)
                else:
                    pass
    except:
        pass

print(kr_santa_list)

---------------------------------------------------- 091160 ----------------------------------------------------
           1d_pct         Volume
count  119.000000     119.000000
mean     0.196080   48169.907563
std      1.382028   89067.827083
min     -3.870773      57.000000
25%     -0.741290    2937.000000
50%      0.435820    8577.000000
75%      1.129568   39741.500000
max      3.196610  519316.000000
            1d_pct         Volume
count  3984.000000    3984.000000
mean     -0.067096   44943.771586
std       1.730846   83789.398440
min     -17.841058       2.000000
25%      -0.926203    2606.500000
50%      -0.047096    7742.000000
75%       0.781660   49013.250000
max      15.407855  908050.000000
0.016385534108539955
           1d_pct         Volume
count  119.000000     119.000000
mean     0.114116   51300.142857
std      2.014592  118226.516842
min     -5.224755      83.000000
25%     -1.045098    1805.000000
50%     -0.034603    6717.000000
75%      0.998713   33162.000000

          1d_pct        Volume
count  98.000000  9.800000e+01
mean   -0.049478  1.596577e+05
std     1.830596  4.713681e+05
min   -10.592619  1.041000e+03
25%    -0.945037  1.539650e+04
50%     0.185829  3.759600e+04
75%     1.133234  1.253648e+05
max     3.770950  4.395403e+06
            1d_pct        Volume
count  3174.000000  3.174000e+03
mean     -0.009347  1.391000e+05
std       1.707640  2.379445e+05
min     -15.168897  5.000000e+01
25%      -0.867807  1.472700e+04
50%       0.031837  4.308400e+04
75%       0.896090  1.525038e+05
max      15.397382  3.738318e+06
0.6672080398938782
          1d_pct         Volume
count  98.000000      98.000000
mean   -0.231981   87011.632653
std     1.746100  103139.488497
min    -5.844646    1271.000000
25%    -1.185919   14245.250000
50%    -0.253712   38936.000000
75%     0.738820  129185.000000
max     7.902798  408582.000000
            1d_pct        Volume
count  2789.000000  2.789000e+03
mean     -0.003477  1.458994e+05
std       1.731364

           1d_pct         Volume
count  105.000000     105.000000
mean    -0.004282   15181.019048
std      1.882237   20646.697951
min     -5.051713      20.000000
25%     -0.949094    3076.000000
50%      0.041960    6963.000000
75%      0.995974   16717.000000
max      8.369284  108145.000000
            1d_pct         Volume
count  3526.000000    3526.000000
mean     -0.142675   21809.574022
std       2.115793   35752.795052
min     -15.128109       3.000000
25%      -1.211288    3804.250000
50%      -0.120359    8880.500000
75%       0.949145   23908.500000
max      23.239515  357386.000000
0.6411424233818337
           1d_pct         Volume
count  105.000000     105.000000
mean    -0.412548   19263.800000
std      2.317190   27569.220214
min    -15.128109     156.000000
25%     -1.316482    2557.000000
50%     -0.322170    8599.000000
75%      0.710985   24413.000000
max      4.099863  165224.000000
            1d_pct         Volume
count  3115.000000    3115.000000
mean     -0.1

          1d_pct         Volume
count  84.000000      84.000000
mean    0.107482   30305.095238
std     1.354164   57532.240973
min    -3.512484      88.000000
25%    -0.736338    1568.250000
50%     0.118049    7219.000000
75%     0.947598   34668.750000
max     3.896663  360458.000000
            1d_pct         Volume
count  2817.000000    2817.000000
mean           inf   33864.279375
std            NaN   66675.363647
min     -14.346591       0.000000
25%      -0.778643    1614.000000
50%       0.031046    7958.000000
75%       0.782718   35911.000000
max            inf  855558.000000
0.6712215250993905
          1d_pct         Volume
count  84.000000      84.000000
mean   -0.158686   36565.452381
std     1.452738   72001.912615
min    -5.262050      35.000000
25%    -1.107670    1031.250000
50%    -0.271488    7830.500000
75%     0.761482   29879.250000
max     3.900397  368768.000000
            1d_pct         Volume
count  2484.000000    2484.000000
mean           inf   34836.4847

          1d_pct         Volume
count  84.000000      84.000000
mean   -0.129750   22588.202381
std     0.992402   41457.113928
min    -3.104693       4.000000
25%    -0.794756     727.000000
50%    -0.014934    4172.000000
75%     0.486191   27977.750000
max     2.488286  243600.000000
            1d_pct         Volume
count  2817.000000    2817.000000
mean      0.069042   31642.859780
std       1.341759   64014.445803
min     -15.394262       0.000000
25%      -0.544425     837.000000
50%       0.068050    4160.000000
75%       0.694444   32141.000000
max      15.017631  743121.000000
0.13905819578461565
          1d_pct         Volume
count  84.000000      84.000000
mean    0.045326   23835.011905
std     0.898357   46247.285337
min    -2.216511       0.000000
25%    -0.393041     349.750000
50%     0.034741    1868.500000
75%     0.445252   23732.750000
max     2.397004  240222.000000
            1d_pct         Volume
count  2484.000000    2484.000000
mean      0.062209   32233.929

In [85]:
for i in kr_santa_list:
    try : 
        print(f'---------------------------------------------------- {i} ----------------------------------------------------')
        df = fdr.DataReader(i, start='2000-01-01', end='2023-02-01')
        p_value1 = mannwhitneyu_test(df)
        p_value_list = mannwhitneyu_test_preemptive_reflection(df)
    except:
        pass

---------------------------------------------------- 091160 ----------------------------------------------------
           1d_pct         Volume
count  119.000000     119.000000
mean     0.196080   48169.907563
std      1.382028   89067.827083
min     -3.870773      57.000000
25%     -0.741290    2937.000000
50%      0.435820    8577.000000
75%      1.129568   39741.500000
max      3.196610  519316.000000
            1d_pct         Volume
count  3984.000000    3984.000000
mean     -0.067096   44943.771586
std       1.730846   83789.398440
min     -17.841058       2.000000
25%      -0.926203    2606.500000
50%      -0.047096    7742.000000
75%       0.781660   49013.250000
max      15.407855  908050.000000
0.016385534108539955
           1d_pct         Volume
count  119.000000     119.000000
mean     0.114116   51300.142857
std      2.014592  118226.516842
min     -5.224755      83.000000
25%     -1.045098    1805.000000
50%     -0.034603    6717.000000
75%      0.998713   33162.000000

In [86]:
kr_santa_list

['091160', '117460']

In [87]:
us_santa_list = []

for i in us_etf_list:
    try : 
        print(f'---------------------------------------------------- {i} ----------------------------------------------------')
        df = fdr.DataReader(i, start='2000-01-01', end='2023-02-01')
        p_value1 = mannwhitneyu_test(df)
        p_value_list = mannwhitneyu_test_preemptive_reflection(df)
        if round(p_value1, 5) <= 0.05:
            us_santa_list.append(i)
        else:
            for item in p_value_list:
                if round(item, 5) <= 0.05:
                    us_santa_list.append(i)
                else:
                    pass
    except:
        pass

print(us_santa_list)

---------------------------------------------------- SOXX ----------------------------------------------------
           1d_pct        Volume
count  154.000000  1.540000e+02
mean     0.009058  2.814364e+05
std      1.429185  3.049309e+05
min     -3.635303  1.500000e+03
25%     -0.770602  6.490000e+04
50%     -0.042068  1.583500e+05
75%      0.580984  3.939250e+05
max      5.673759  1.411300e+06
            1d_pct        Volume
count  5268.000000  5.268000e+03
mean     -0.008528  4.084723e+05
std       1.692881  4.559302e+05
min      -8.800235  3.000000e+02
25%      -0.892415  1.106000e+05
50%       0.002985  2.542500e+05
75%       0.889045  5.359250e+05
max      10.777773  5.957900e+06
0.6552687403729613
           1d_pct        Volume
count  154.000000  1.540000e+02
mean    -0.004547  3.635558e+05
std      1.686738  3.708505e+05
min     -6.264416  6.300000e+03
25%     -0.892413  9.070000e+04
50%     -0.031472  2.411500e+05
75%      0.832493  4.726000e+05
max      5.756933  1.661700e+

           1d_pct        Volume
count  119.000000  1.190000e+02
mean     0.013596  1.552236e+06
std      1.461325  1.386308e+06
min     -4.563375  2.630000e+04
25%     -0.654315  4.073500e+05
50%      0.075930  1.234900e+06
75%      0.707629  2.231750e+06
max      4.442251  7.426300e+06
            1d_pct        Volume
count  4095.000000  4.095000e+03
mean     -0.019589  2.410920e+06
std       2.087229  2.135007e+06
min     -12.165450  1.000000e+02
25%      -1.022487  6.820500e+05
50%       0.000000  2.065800e+06
75%       0.982813  3.376200e+06
max      15.410200  2.204690e+07
0.7825137595540326
           1d_pct        Volume
count  119.000000  1.190000e+02
mean    -0.103111  2.027107e+06
std      1.980472  1.754417e+06
min     -9.808343  4.320000e+04
25%     -0.778603  5.413500e+05
50%      0.079812  1.878600e+06
75%      0.899286  2.819550e+06
max      7.188841  9.467900e+06
            1d_pct        Volume
count  3633.000000  3.633000e+03
mean     -0.034711  2.464154e+06
std      

           1d_pct        Volume
count  161.000000  1.610000e+02
mean    -0.006131  5.466236e+06
std      1.149295  5.282124e+06
min     -4.354354  3.400000e+03
25%     -0.554601  8.064000e+05
50%     -0.036101  4.730300e+06
75%      0.380379  7.471500e+06
max      6.745182  2.597620e+07
            1d_pct        Volume
count  5646.000000  5.646000e+03
mean     -0.007827  8.509629e+06
std       1.123709  7.710653e+06
min      -8.221797  4.000000e+02
25%      -0.562049  9.648000e+05
50%       0.031662  8.206050e+06
75%       0.579545  1.258932e+07
max       7.958120  7.911820e+07
0.32003301659069816
           1d_pct        Volume
count  161.000000  1.610000e+02
mean     0.011573  7.994956e+06
std      1.138057  6.578869e+06
min     -7.474111  8.000000e+02
25%     -0.402820  1.097900e+06
50%      0.027622  7.695300e+06
75%      0.537781  1.176770e+07
max      4.279978  2.785590e+07
            1d_pct        Volume
count  5028.000000  5.028000e+03
mean     -0.006469  8.565733e+06
std     

          1d_pct         Volume
count  84.000000      84.000000
mean    0.008655   61529.761905
std     1.077965  105378.574878
min    -2.767300       0.000000
25%    -0.535982    9150.000000
50%    -0.015201   25000.000000
75%     0.464436   52425.000000
max     4.329935  588000.000000
            1d_pct        Volume
count  2939.000000  2.939000e+03
mean     -0.053514  5.263161e+04
std       1.262602  9.838648e+04
min      -9.617119  0.000000e+00
25%      -0.709260  1.030000e+04
50%       0.000000  2.490000e+04
75%       0.655003  5.590000e+04
max      16.293284  1.619000e+06
0.9686485810598948
          1d_pct        Volume
count  84.000000  8.400000e+01
mean   -0.066547  6.581548e+04
std     0.826375  1.302362e+05
min    -2.354983  8.000000e+02
25%    -0.608007  8.650000e+03
50%    -0.017314  2.670000e+04
75%     0.432077  7.585000e+04
max     2.297607  1.006400e+06
            1d_pct        Volume
count  2607.000000  2.607000e+03
mean     -0.055628  5.013161e+04
std       1.233956

           1d_pct        Volume
count  126.000000  1.260000e+02
mean     0.033773  3.837016e+05
std      0.924125  5.374953e+05
min     -2.620610  0.000000e+00
25%     -0.461632  1.220250e+05
50%      0.000000  2.521500e+05
75%      0.417287  4.624500e+05
max      3.759114  5.124600e+06
            1d_pct        Volume
count  4205.000000  4.205000e+03
mean      0.004066  7.249806e+05
std       1.484590  9.811247e+05
min     -16.798077  0.000000e+00
25%      -0.540312  1.562000e+05
50%       0.048914  3.556000e+05
75%       0.559401  8.850000e+05
max      16.032059  9.463800e+06
0.6315292857597623
           1d_pct        Volume
count  125.000000  1.250000e+02
mean    -0.079707  7.930768e+05
std      1.797241  1.290681e+06
min    -10.603363  3.000000e+02
25%     -0.479063  2.013000e+05
50%      0.030351  3.897000e+05
75%      0.394212  8.433000e+05
max      8.580013  9.241200e+06
            1d_pct        Volume
count  3718.000000  3.718000e+03
mean      0.006388  7.306521e+05
std      

In [88]:
us_santa_list

['QQQ', 'XTN', 'KIE']

In [89]:
for i in us_santa_list:
    try : 
        print(f'---------------------------------------------------- {i} ----------------------------------------------------')
        df = fdr.DataReader(i, start='2000-01-01', end='2023-02-01')
        p_value1 = mannwhitneyu_test(df)
        p_value_list = mannwhitneyu_test_preemptive_reflection(df)
    except:
        pass

---------------------------------------------------- QQQ ----------------------------------------------------
           1d_pct        Volume
count  161.000000  1.610000e+02
mean     0.114357  4.562047e+07
std      2.094117  2.868743e+07
min     -8.751334  7.089000e+06
25%     -0.604212  2.489050e+07
50%     -0.037737  3.824040e+07
75%      0.556756  5.761270e+07
max     19.640719  1.676895e+08
            1d_pct        Volume
count  5646.000000  5.646000e+03
mean     -0.008656  6.985502e+07
std       1.473533  5.037099e+07
min      -9.534425  5.212000e+06
25%      -0.606054  3.234550e+07
50%       0.062658  5.872265e+07
75%       0.659470  9.198472e+07
max      14.880833  6.167723e+08
0.5391922367664107
           1d_pct        Volume
count  161.000000  1.610000e+02
mean     0.047176  7.555018e+07
std      1.401624  6.539821e+07
min     -5.422532  1.252240e+07
25%     -0.505301  3.110120e+07
50%      0.062737  6.171540e+07
75%      0.608179  8.881400e+07
max      6.466263  3.424724e+0

+ 한국 ETF의 경우 반도체와 에너지 화학 (모두 산타랠리 기간에 상승)
+ 미국 ETF의 경우 기술주, 운송, 보험 (모두 산타랠리 기간전에 하락)

In [90]:
index = [ks, ks200, kq, nq, sp, dow]
for i in index:
    print('---------------------------------------------------------------------------------')
    p_value1 = mannwhitneyu_test(i)
    p_value_list = mannwhitneyu_test_preemptive_reflection(df)

---------------------------------------------------------------------------------
           1d_pct        Volume
count  161.000000  1.610000e+02
mean    -0.001272  4.110348e+05
std      1.105579  2.109544e+05
min     -3.760335  1.477000e+05
25%     -0.594009  2.832000e+05
50%      0.067491  3.590000e+05
75%      0.501923  4.580000e+05
max      3.861033  1.519900e+06
            1d_pct        Volume
count  5531.000000  5.531000e+03
mean     -0.038983  4.759988e+05
std       1.148376  2.641589e+05
min     -10.729565  1.363000e+05
25%      -0.558011  3.153000e+05
50%      -0.023817  4.024000e+05
75%       0.521659  5.430000e+05
max       8.977477  3.455500e+06
0.8412440936361698
           1d_pct        Volume
count  125.000000  1.250000e+02
mean    -0.079707  7.930768e+05
std      1.797241  1.290681e+06
min    -10.603363  3.000000e+02
25%     -0.479063  2.013000e+05
50%      0.030351  3.897000e+05
75%      0.394212  8.433000e+05
max      8.580013  9.241200e+06
            1d_pct        

           1d_pct        Volume
count  161.000000  1.610000e+02
mean     0.125436  1.897030e+09
std      1.779419  1.173393e+09
min     -7.368150  4.909900e+08
25%     -0.495712  1.219050e+09
50%     -0.023784  1.515600e+09
75%      0.555550  2.186830e+09
max     16.062109  6.971860e+09
            1d_pct        Volume
count  5647.000000  5.647000e+03
mean     -0.006284  2.313768e+09
std       1.325517  1.100941e+09
min      -7.676289  2.214300e+08
25%      -0.589515  1.734100e+09
50%       0.069788  1.958250e+09
75%       0.638487  2.314640e+09
max       9.471873  1.162119e+10
0.6310190619755061
           1d_pct        Volume
count  125.000000  1.250000e+02
mean    -0.079707  7.930768e+05
std      1.797241  1.290681e+06
min    -10.603363  3.000000e+02
25%     -0.479063  2.013000e+05
50%      0.030351  3.897000e+05
75%      0.394212  8.433000e+05
max      8.580013  9.241200e+06
            1d_pct        Volume
count  3718.000000  3.718000e+03
mean      0.006388  7.306521e+05
std      

## ols 검증

In [41]:
def ols_test(df):
    santa_period = sep_date(df)
    santa_period = pd.concat(santa_period).index

    df = make_1d_pct(df)
    df['SantaClausRally'] = 0
    df.loc[df.index.isin(santa_period), 'SantaClausRally'] = 1
    
    # 독립 변수와 종속 변수 설정
    y = df['1d_pct']  # 종속 변수
    X = df['SantaClausRally']  # 독립 변수

    # 절편을 추가하기 위해 X에 상수항을 추가합니다.
    X = sm.add_constant(X)

    # OLS(최소제곱법)를 사용하여 선형 회귀 모델 피팅
    model = sm.OLS(y, X).fit()

    # 회귀분석 결과 요약
    print(model.summary())

In [73]:
ols_test(dow)

                            OLS Regression Results                            
Dep. Variable:                 1d_pct   R-squared:                       0.000
Model:                            OLS   Adj. R-squared:                 -0.001
Method:                 Least Squares   F-statistic:                   0.07593
Date:                Sun, 31 Dec 2023   Prob (F-statistic):              0.783
Time:                        00:38:27   Log-Likelihood:                -2001.9
No. Observations:                 975   AIC:                             4008.
Df Residuals:                     973   BIC:                             4018.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                      coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------
const              -0.0307      0.061     

In [52]:
def ols_test_preemptive_reflection(df):
    reflection_range = [[-33, -26], [-26, -19], [-19, -12], [-12, -5]]
    p_value_list = []
    for i in reflection_range:
        santa_period, rest_ = preemptive_reflection(df, i[0], i[1])
        santa_period = pd.concat(santa_period).index
        df = make_1d_pct(df)
        df['SantaClausRally'] = 0
        df.loc[df.index.isin(santa_period), 'SantaClausRally'] = 1

        # 독립 변수와 종속 변수 설정
        y = df['1d_pct']  # 종속 변수
        X = df['SantaClausRally']  # 독립 변수

        # 절편을 추가하기 위해 X에 상수항을 추가합니다.
        X = sm.add_constant(X)

        # OLS(최소제곱법)를 사용하여 선형 회귀 모델 피팅
        model = sm.OLS(y, X).fit()

        # 회귀분석 결과 요약
        print(model.summary())
    return p_value_list

In [74]:
ols_test_preemptive_reflection(dow)

                            OLS Regression Results                            
Dep. Variable:                 1d_pct   R-squared:                       0.000
Model:                            OLS   Adj. R-squared:                 -0.001
Method:                 Least Squares   F-statistic:                    0.3476
Date:                Sun, 31 Dec 2023   Prob (F-statistic):              0.556
Time:                        00:38:32   Log-Likelihood:                -2001.8
No. Observations:                 975   AIC:                             4008.
Df Residuals:                     973   BIC:                             4017.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                      coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------
const              -0.0339      0.061     

[]

## 종목 불러오기

In [7]:
# ks_stock_list = fdr.StockListing('KOSPI')
# ks_code_list = ks_stock_list['Code'].apply(lambda x : str(x) + '.KS').tolist()

In [8]:
# ks200_html = pd.read_html('https://ko.wikipedia.org/wiki/%EC%BD%94%EC%8A%A4%ED%94%BC_200')[-1]
# ks200_html.columns = ['Sector', 'Name']
# # print(ks200_html.columns)
# ks200 = ks200_html.iloc[1:,]
# ks200.columns = ['sector', 'stock']
# ks200_stock_name = []
# for i in range(1, 12):
#     data = ks200['stock'][i].split()
#     sector_name = ks200_html['Sector'][i]
#     for j in data:
#         dicts = {'Name' : j, 'sector' : sector_name}
#         ks200_stock_name.append(dicts)
# ks200 = pd.DataFrame(ks200_stock_name)

In [9]:
# # 시장별 데이터 가져오기 : 'KOSPI', 'KODAQ', 'KONEX'
# stocks_kospi = fdr.StockListing('KOSPI')[['Code', 'Name']]
# # 앞 5개 뒷부분 5개 보여준다.
# ks200 = pd.merge(ks200, stocks_kospi, on='Name')
# ks200['Code'] = ks200['Code'].apply(lambda x : str(x) + '.KS')

In [10]:
# santa_list = []
# for i in ks_code_list:
#     try : 
#         print(f'---------------------------------------------------- {i} ----------------------------------------------------')
#         df = yf.download(i, start='2000-01-01', end='2023-02-01')
#         p_value1 = mannwhitneyu_test(df)
#         p_value_list = mannwhitneyu_test_preemptive_reflection(df)
#         if round(p_value1, 5) <= 0.05:
#             santa_list.append(i)
#         else:
#             for item in p_value_list:
#                 print(item)
#                 if round(item, 5) <= 0.05:
#                     santa_list.append(i)
#                 else:
#                     pass
#     except:
#         pass

---------------------------------------------------- 375500.KS ----------------------------------------------------
[*********************100%***********************]  1 of 1 completed
0.42938093361928564
0.9068011839763648
0.6254618328601236
0.8384036229871814
---------------------------------------------------- 006360.KS ----------------------------------------------------
[*********************100%***********************]  1 of 1 completed
0.32789344893555916
0.7788754070884185
0.45367981161522064
0.8615219414429245
---------------------------------------------------- 047040.KS ----------------------------------------------------
[*********************100%***********************]  1 of 1 completed
0.4255148521533041
0.8703786408644023
0.9417156377087438
0.532359589665377
---------------------------------------------------- 028260.KS ----------------------------------------------------
[*********************100%***********************]  1 of 1 completed
0.8488507331892882
0.350081120

[*********************100%***********************]  1 of 1 completed
0.47206095549735305
0.7414370971627533
0.5796613968705402
0.27703511785446877
---------------------------------------------------- 011200.KS ----------------------------------------------------
[*********************100%***********************]  1 of 1 completed
---------------------------------------------------- 373220.KS ----------------------------------------------------
[*********************100%***********************]  1 of 1 completed
0.18227511753792858
0.8573051073431218
0.0370502965504125
0.6971786934041178
---------------------------------------------------- 079550.KS ----------------------------------------------------
[*********************100%***********************]  1 of 1 completed
---------------------------------------------------- 006260.KS ----------------------------------------------------
[*********************100%***********************]  1 of 1 completed
0.6350995573162349
0.850868923082690

[*********************100%***********************]  1 of 1 completed
0.5336735177920516
0.6530204326015161
0.31109299272736635
0.4979606681707588
---------------------------------------------------- 003230.KS ----------------------------------------------------
[*********************100%***********************]  1 of 1 completed
0.9638483500599995
0.9423328138354464
0.8834943977491869
0.6407774482553688
---------------------------------------------------- 002790.KS ----------------------------------------------------
[*********************100%***********************]  1 of 1 completed
0.9850986475998652
0.06992025053631995
0.5753626876095217
0.12463702899260468
---------------------------------------------------- 090430.KS ----------------------------------------------------
[*********************100%***********************]  1 of 1 completed
0.9480017186687191
0.6230611991623372
0.6903637827859734
0.527031492018246
---------------------------------------------------- 007310.KS -------

[*********************100%***********************]  1 of 1 completed
0.37661997364863664
0.9545515552146875
0.6611215816357963
0.362818511060533
---------------------------------------------------- 120110.KS ----------------------------------------------------
[*********************100%***********************]  1 of 1 completed
---------------------------------------------------- 014680.KS ----------------------------------------------------
[*********************100%***********************]  1 of 1 completed
0.6424007764325277
0.23353449265166648
0.726949462046359
0.030400303594211975
---------------------------------------------------- 000880.KS ----------------------------------------------------
[*********************100%***********************]  1 of 1 completed
0.7795370296661195
0.8832636929923721
0.5828461702910122
0.9764883035162465
---------------------------------------------------- 009830.KS ----------------------------------------------------
[*********************100%****

[*********************100%***********************]  1 of 1 completed
0.07370143073656273
0.9781840524177371
0.6324638000072016
0.9407588759268164
---------------------------------------------------- 017800.KS ----------------------------------------------------
[*********************100%***********************]  1 of 1 completed
---------------------------------------------------- 016380.KS ----------------------------------------------------
[*********************100%***********************]  1 of 1 completed
---------------------------------------------------- 005490.KS ----------------------------------------------------
[*********************100%***********************]  1 of 1 completed
0.3726339207069229
0.11171334182837918
0.014122891377679548
0.2684063819333661
---------------------------------------------------- 002710.KS ----------------------------------------------------
[*********************100%***********************]  1 of 1 completed
0.08133264721031332
0.4072937109060

In [12]:
# for i in ks200['Code'].tolist():
#     print(f'---------------------------------------------------- {i} ----------------------------------------------------')
#     df = yf.download(i, start='2000-01-01', end='2023-02-01')
#     mannwhitneyu_test(df)
#     mannwhitneyu_test_preemptive_reflection(df)